In [42]:
# Import necessary libraries

import json
import joblib
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/agastimhatre/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Step 1: Join business dataset with reviews dataset. Tokenize reviews and convert to lowercase

In [43]:
# Store reviews for each business
business_information = defaultdict(lambda: {"reviews": []})

with open("yelp_academic_dataset_review.json", "r", encoding="utf-8") as file:

    for line in file: 

        curr = json.loads(line)
        business_information[curr["business_id"]]["reviews"].append(curr["text"])

In [44]:
# Extract business-specific data and store in dictionary
with open("yelp_academic_dataset_business.json", "r", encoding="utf-8") as file:

    for line in file:

        curr = json.loads(line)
        business_information[curr["business_id"]]["name"] = curr["name"]
        business_information[curr["business_id"]]["address"] = curr["address"]
        business_information[curr["business_id"]]["city"] = curr["city"]
        business_information[curr["business_id"]]["state"] = curr["state"]


In [ ]:
# Tokenize reviews and convert all tokens to lowercase
tokenized_business_information = defaultdict(lambda: {"reviews": []})

for id_ in business_information.keys():

    for review in business_information[id_]["reviews"]:

        tokenized_business_information[id_]["reviews"].append([word.lower() for word in word_tokenize(review)])

In [ ]:
# Write joined, tokenized business/review data to a file
with open('business_with_reviews.json', 'w') as f:
    for key, value in tokenized_business_information.items():
        json.dump({key: value}, f)
        f.write('\n')

## Step 2: Output scores for TFIDF ranker

In [45]:
# Read in business reviews from 
business_reviews = dict()

with open('business_with_reviews.json', 'r') as f:

    for line in f:

        record = json.loads(line)
        business_reviews.update(record)

In [ ]:
# Test one query on the entire dataset, check results

business_ids = list(business_reviews.keys())
corpus = []
for id_ in business_ids:

    corpus.append(" ".join(" ".join(tokens) for tokens in business_reviews[id_]["reviews"]))

In [ ]:
# Create TFIDF vector and train vectorizer
# Persist vector and model 
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')
scipy.sparse.save_npz("tfidf_matrix.npz", tfidf_matrix)

In [46]:
# Reinitialize vectorizer and tfidf_matrix if session is lost

vectorizer = joblib.load('tfidf_vectorizer.joblib')
tfidf_matrix = scipy.sparse.load_npz("tfidf_matrix.npz")

In [47]:
query = "cafe with great lattes in san francisco"
query_vec = vectorizer.transform([query])
similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

In [ ]:
scores = []

n = len(business_ids)
for i in range(n): scores.append((business_ids[i], similarities[i]))

scores.sort(key = lambda x: x[1], reverse=True)

In [ ]:
for id, _ in scores[:10]:

    print(business_information[id]["name"])

## Step 3: Create/Store Embeddings and Use FAISS to generate scores, Compare to token-matching ranker